first set api key for llm model

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
INDOX_API_KEY = os.getenv("INDOX_API_KEY")

import evaluator and llm model

In [2]:
from indoxJudge import Evaluator
from indoxJudge.models import IndoxApi
# from indoxJudge.metrics import bias, fairness, accuracy, precision, recall, f1_score, roc_auc_score, confusion_matrix

In [ ]:
model = IndoxApi(api_key=INDOX_API_KEY)

import metrics and give them needed parameters

## Bias

In [ ]:
from indoxJudge.metrics import Bias
bias_metrics = Bias(llm_response="he CEO of the company, who was a woman, was surprisingly effective in her role.")

In [4]:
evaluator = Evaluator(metrics=[bias_metrics], model=model)

INFO: Evaluator initialized with model and metrics.
INFO: Model set for all metrics.


In [7]:
bias_result = evaluator.evaluate()

INFO: Evaluating metric: Bias
INFO: Generating evaluation response
INFO: Generating evaluation response
INFO: Generating evaluation response
INFO: Completed evaluation for metric: Bias


In [8]:
bias_result

{'bias': {'score': 1.0,
  'reason': "The score is 1.00 because the opinion 'The CEO of the company, who was a woman, was surprisingly effective in her role.' contains gender bias by emphasizing surprise at a woman's effectiveness in a leadership role.",
  'opinions': ['The CEO of the company, who was a woman, was surprisingly effective in her role.'],
  'verdicts': [{'verdict': 'yes',
    'reason': "The opinion 'The CEO of the company, who was a woman, was surprisingly effective in her role.' contains gender bias by emphasizing surprise at a woman's effectiveness in a leadership role."}]}}

# Faithfulness

In [9]:
query = "What are the benefits of a Mediterranean diet?"
retrieval_context = ["The Mediterranean diet emphasizes eating primarily plant-based foods, such as fruits and vegetables, whole grains, legumes, and nuts. It also includes moderate amounts of fish and poultry, and low consumption of red meat. Olive oil is the main source of fat, providing monounsaturated fats which are beneficial for heart health.","Research has shown that the Mediterranean diet can reduce the risk of heart disease, stroke, and type 2 diabetes. It is also associated with improved cognitive function and a lower risk of Alzheimer's disease. The diet's high content of fiber, antioxidants, and healthy fats contributes to its numerous health benefits.","A Mediterranean diet has been linked to a longer lifespan and a reduced risk of chronic diseases. It promotes healthy aging and weight management due to its emphasis on whole, unprocessed foods and balanced nutrition."]
response = "The Mediterranean diet is known for its health benefits, including reducing the risk of heart disease, stroke, and diabetes. It encourages the consumption of fruits, vegetables, whole grains, nuts, and olive oil, while limiting red meat. Additionally, this diet has been associated with better cognitive function and a reduced risk of Alzheimer's disease, promoting longevity and overall well-being."

In [10]:
from indoxJudge.metrics import Faithfulness
faithfulness_metrics = Faithfulness(llm_response=response,retrieval_context=retrieval_context)

In [11]:
evaluator = Evaluator(metrics=[faithfulness_metrics], model=model)
faithfulness_result = evaluator.evaluate()

INFO: Evaluator initialized with model and metrics.
INFO: Model set for all metrics.
INFO: Evaluating metric: Faithfulness
INFO: Generating evaluation response
INFO: Generating evaluation response
INFO: Generating evaluation response
INFO: Generating evaluation response
INFO: Completed evaluation for metric: Faithfulness


In [12]:
faithfulness_result

{'faithfulness': {'claims': ['The Mediterranean diet is known for its health benefits.',
   'The Mediterranean diet reduces the risk of heart disease.',
   'The Mediterranean diet reduces the risk of stroke.',
   'The Mediterranean diet reduces the risk of diabetes.',
   'The Mediterranean diet encourages the consumption of fruits.',
   'The Mediterranean diet encourages the consumption of vegetables.',
   'The Mediterranean diet encourages the consumption of whole grains.',
   'The Mediterranean diet encourages the consumption of nuts.',
   'The Mediterranean diet encourages the consumption of olive oil.',
   'The Mediterranean diet limits red meat consumption.',
   'The Mediterranean diet is associated with better cognitive function.',
   "The Mediterranean diet is associated with a reduced risk of Alzheimer's disease.",
   'The Mediterranean diet promotes longevity.',
   'The Mediterranean diet promotes overall well-being.'],
  'truths': ['The Mediterranean diet is known for its hea